In [1]:
import sys
import os
import numpy as np
from tqdm import tqdm
import pickle

# Get the parent folder path (folder K)
parent_folder = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(parent_folder)

# Add folder A to the Python path
folder_classes_path = os.path.join(parent_folder, 'classes')
sys.path.append(folder_classes_path)

/Users/zeyichen/Github/Repo/WB_Algo/Stochastic_FP


Load necessary python classes

In [2]:
from true_WB import *
from input_generate_plugin import *
from entropic_estimate_OT import *
from entropic_iterative_scheme import *
from config_log import *
from measure_visualize import *
from input_generate_entropy import *
from sample_plot import *

# Sampler set-up

In [3]:
dim = 2
num_samples = 5000
num_measures = 5
iter = 0
truncated_radius = 150
seed = 1009

In [4]:
source_sampler = MixtureOfGaussians(dim)
source_sampler.random_components(num_components=5, uniform_weights = True, seed = seed) # seed from the measure selection
# source_sampler.set_truncation(truncated_radius)

In [5]:
auxiliary_measure_sampler_set = []
for auxiliary_seed in [1010, 1018, 1014, 1016, 1003]:
    auxiliary_measure_sampler = MixtureOfGaussians(dim)
    auxiliary_measure_sampler.random_components(num_components=5, uniform_weights = True, seed = auxiliary_seed)
    # auxiliary_measure_sampler.set_truncation(truncated_radius)
    auxiliary_measure_sampler_set.append(auxiliary_measure_sampler)

In [7]:
# construct entropic_sampler
entropic_sampler = entropic_input_sampler(dim, 
                                          num_measures, 
                                          auxiliary_measure_sampler_set, 
                                          source_sampler = source_sampler, 
                                          n_k = 1000, 
                                          seed = 120, 
                                          gamma = 0.3, 
                                          manual = True,
                                          truncated_radius = truncated_radius,
                                          bound_type = "eigen_bound")

In [8]:
entropic_sampler.__dict__

{'dim': 2,
 'num_measures': 5,
 'auxiliary_measure_sampler_set': [<true_WB.MixtureOfGaussians at 0x17c571d50>,
 'tilde_K': 5,
 'source_sampler': <true_WB.MixtureOfGaussians at 0x147ff4fd0>,
 'n_k': 1000,
 'seed': 120,
 'rng_entropy': RandomState(MT19937) at 0x17E0A9040,
 'gamma': 0.3,
 'manual': True,
 'truncated_radius': 150,
 'bound_type': 'eigen_bound',
 'grid_size': 200}

# Entropic Sampler Set-up

Only for entropic sampler configuration at the first time. Jump to "Set-up over" section if the configuration is already done.

In [ ]:
# generate strong convexity parameters of the mappings.
entropic_sampler.generate_strong_convexity_param()
print("strong convexity parameters all set.")

In [ ]:
# assign theta
entropic_sampler.assign_theta()
print("theta all set.")

In [ ]:
# generate Y matrices
entropic_sampler.generate_Y_matrices()
print("Y matrices all set.")

In [ ]:
# generate g vectors
entropic_sampler.generate_g_vectors()
print("g vectors all set.")

In [ ]:
# generate smoothness parameters; this involves solving max eigen for each tilde_k
entropic_sampler.generate_smoothness_param()
print("smoothness parameters all set.")

In [ ]:
print(entropic_sampler.smoothness_param_dict)

In [ ]:
# construct a surjective mapping to map component maps to their respective OT maps for generating input measures.
entropic_sampler.construct_surjective_mapping()
print("surjective mapping all set.")

In [ ]:
# generate A matrices
entropic_sampler.generate_A_matrices()
print("A matrices all set.")

In [8]:
epsilon = 10
result_dir = "results"
os.makedirs(result_dir, exist_ok=True)
save_pathname = f"{result_dir}/entropic_measures_{num_measures}_seed_{seed}_samples_{num_samples}_dim_{dim}_epsilon_{epsilon}"
os.makedirs(save_pathname, exist_ok=True)

In [9]:
import pickle

with open(f"{save_pathname}/entropic_sampler_info.pkl", "wb") as f:
    pickle.dump(entropic_sampler.__dict__, f)

with open(f"{save_pathname}/source_sampler_info.pkl", "wb") as f:
    pickle.dump(source_sampler.__dict__, f)

print(f"Data successfully saved")

Data successfully saved


# Set-up Over

In [9]:
epsilon = 10
result_dir = "results"
os.makedirs(result_dir, exist_ok=True)
save_pathname = f"{result_dir}/entropic_measures_{num_measures}_seed_{seed}_samples_{num_samples}_dim_{dim}_epsilon_{epsilon}"
os.makedirs(save_pathname, exist_ok=True)

In [10]:
# Load the sampler attributes
with open(f"{save_pathname}/entropic_sampler_info.pkl", "rb") as f:
    loaded_data_entropic_sampler = pickle.load(f)

with open(f"{save_pathname}/source_sampler_info.pkl", "rb") as f:
    loaded_data_source_sampler = pickle.load(f)

print("Loaded Data for Entropic Sampler:", loaded_data_entropic_sampler)
print("Loaded Data for Source Sampler:", loaded_data_source_sampler)

Loaded Data for Entropic Sampler: {'dim': 2, 'num_measures': 5, 'auxiliary_measure_sampler_set': [<true_WB.MixtureOfGaussians object at 0x17e0e7c50>, <true_WB.MixtureOfGaussians object at 0x17e0f4bd0>, <true_WB.MixtureOfGaussians object at 0x17e0f5050>, <true_WB.MixtureOfGaussians object at 0x17e0f54d0>, <true_WB.MixtureOfGaussians object at 0x17e0f5950>], 'tilde_K': 5, 'source_sampler': <true_WB.MixtureOfGaussians object at 0x17e0f5e90>, 'n_k': 1000, 'seed': 120, 'rng_entropy': RandomState(MT19937) at 0x17E0A9240, 'gamma': 0.3, 'manual': True, 'truncated_radius': 150, 'bound_type': 'eigen_bound', 'grid_size': 200, 'strong_convexity_param_dict': {0: 0.0001, 1: 0.0001, 2: 0.0001, 3: 0.0001, 4: 0.0001}, 'theta_dict': {0: 10, 1: 10, 2: 10, 3: 10, 4: 10}, 'Y_matrix_dict': {0: array([[ 83.25389778,  -6.77748404],
       [-24.38674772, -23.76266702],
       [ 97.2196175 ,  -0.66185246],
       ...,
       [-35.17059538,  10.05658482],
       [-14.49614928, -84.75519572],
       [-10.84955811

In [11]:
source_sampler = MixtureOfGaussians(dim)
source_sampler.__dict__.update(loaded_data_source_sampler)

entropic_sampler = entropic_input_sampler(dim, num_measures, auxiliary_measure_sampler_set, source_sampler = source_sampler, n_k = 1000, seed = 120)
entropic_sampler.__dict__.update(loaded_data_entropic_sampler)

In [12]:
entropic_sampler.__dict__

{'dim': 2,
 'num_measures': 5,
 'auxiliary_measure_sampler_set': [<true_WB.MixtureOfGaussians at 0x17e0e7c50>,
 'tilde_K': 5,
 'source_sampler': <true_WB.MixtureOfGaussians at 0x17e0f5e90>,
 'n_k': 1000,
 'seed': 120,
 'rng_entropy': RandomState(MT19937) at 0x17E0A9240,
 'gamma': 0.3,
 'manual': True,
 'truncated_radius': 150,
 'bound_type': 'eigen_bound',
 'grid_size': 200,
 'strong_convexity_param_dict': {0: 0.0001,
  1: 0.0001,
  2: 0.0001,
  3: 0.0001,
  4: 0.0001},
 'theta_dict': {0: 10, 1: 10, 2: 10, 3: 10, 4: 10},
 'Y_matrix_dict': {0: array([[ 83.25389778,  -6.77748404],
         [-24.38674772, -23.76266702],
         [ 97.2196175 ,  -0.66185246],
         ...,
         [-35.17059538,  10.05658482],
         [-14.49614928, -84.75519572],
         [-10.84955811, -17.94279971]]),
  1: array([[-16.97145061,  91.46456815],
         [ 98.16619451, -18.46596147],
         [-75.22131842, -52.44109978],
         ...,
         [ 47.97628268,  31.75763179],
         [-78.94388382, -55.91

In [13]:
# Ensure the plot directory exists
plot_dirc = f"{save_pathname}/plots"
os.makedirs(plot_dirc, exist_ok=True)

In [13]:
visualize_num_samples = 1000

# Sample and plot the source measure
source_measure_samples = source_sampler.sample(visualize_num_samples, multiplication_factor=1)
print("Start to plot the source measure")
plot_2d_source_measures_kde(source_measure_samples, plot_dirc=plot_dirc, scatter=False)
print("Finish plotting the source measure")

# Sample input measures
input_measure_samples = entropic_sampler.sample(visualize_num_samples)

# Set up a single row of subplots for input measures
num_measures = len(input_measure_samples)
fig, axes = plt.subplots(1, num_measures, figsize=(5 * num_measures, 5))

if num_measures == 1:
    axes = [axes]  # Ensure axes is iterable when there's only one subplot

for measure_index, ax in enumerate(axes):
    measure_samples = np.array(input_measure_samples[measure_index])
    print(f"Start to plot the input measure {measure_index}")
    
    # Plot the KDE for the input measure
    plot_2d_input_measure_kde_row(measure_samples, measure_index, scatter=False, ax=ax)
    ax.set_title(r"$\widetilde{{\nu}}_{%d}$" % measure_index)
    print(f"Finish plotting the input measure {measure_index}")

# Save the row plot of input measures
plt.tight_layout()
input_plot_path = os.path.join(plot_dirc, "input_measures_row_plot.png")
plt.savefig(input_plot_path)
plt.close()

print(f"All input measures plotted and saved at {input_plot_path}")


source sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

source sampling: 100%|██████████| 1000/1000 [00:00<00:00, 5989.28it/s]


Start to plot the source measure
Finish plotting the source measure


Generating 1000 input measure samples: 100%|██████████| 1000/1000 [00:04<00:00, 235.05it/s]


Start to plot the input measure 0
Finish plotting the input measure 0
Start to plot the input measure 1
Finish plotting the input measure 1
Start to plot the input measure 2
Finish plotting the input measure 2
Start to plot the input measure 3
Finish plotting the input measure 3
Start to plot the input measure 4
Finish plotting the input measure 4
All input measures plotted and saved at results/entropic_measures_5_seed_1009_samples_5000_dim_2_epsilon_10/plots/input_measures_row_plot.png


In [14]:
plot_2d_gmm_pdf(source_sampler, truncated_radius, save_path = plot_dirc, file_name="source_GMM_pdf", title_name = r"$\bar{\mu}$")
# here the truncated radius is used to set grids

In [15]:
i = 0
for auxiliary_measure_sampler in auxiliary_measure_sampler_set:
    plot_2d_gmm_pdf(auxiliary_measure_sampler, truncated_radius, save_path = plot_dirc, file_name=f"auxiliary_{i}_pdf", title_name = r"$\widetilde{{\nu}}_{%d}$" % i)
    i += 1

# Iteration starts

In [15]:
iterative_computer = entropic_iterative_scheme(dim, 
                             num_measures, 
                             source_sampler, 
                             entropic_sampler, 
                             source_sampler_seed = 1009,
                             truncate_radius = truncated_radius,
                             log = True)

In [16]:
source_measure_samples = source_sampler.sample(num_samples, multiplication_factor=1)

source sampling: 100%|██████████| 5000/5000 [00:00<00:00, 9887.53it/s] 


In [17]:
input_measure_samples = entropic_sampler.sample(num_samples)

Generating 5000 input measure samples: 100%|██████████| 5000/5000 [00:25<00:00, 198.53it/s]


In [18]:
iterative_computer.V_value_compute(source_measure_samples, input_measure_samples, iter = None, save_pathname = save_pathname)
print(iterative_computer.V_values)

{'true_V_value': 5398.675178118495}


In [19]:
iterative_computer.converge(source_measure_samples, input_measure_samples, iter, num_samples, max_iter=20, epsilon = epsilon, plot = False, scatter = False, MC_size = 20)

100%|██████████| 5/5 [05:29<00:00, 65.85s/it] 


In [ ]:
with open(f"{save_pathname}/iterative_computer_info.pkl", "wb") as f:
    pickle.dump(iterative_computer.__dict__, f)
    
print(f"Data successfully saved")

# Construct Quantile

In [19]:
from tqdm import tqdm, tqdm_notebook

save_pathname = f"results/entropic_measures_5_seed_1009_samples_5000_dim_2_epsilon_10"

true_V_value_record = []

for i in tqdm(range(200), desc = "Generate source samples for statistical inferences"):
    source_measure_samples = source_sampler.sample(num_samples, multiplication_factor=1)
    input_measure_samples = entropic_sampler.sample(num_samples)
    V_value = 0
    for measure_index in range(num_measures):
        input_samples = np.array(input_measure_samples[measure_index])
        V_value += W2_pot(input_samples, source_measure_samples)
    # normalize the V_value by the number of input measures
    V_value /= num_measures
    true_V_value_record.append(V_value)

    output_file = os.path.join(save_pathname, "true_V_value_record.json")
    with open(output_file, 'w') as f:
        json.dump(true_V_value_record, f, indent=4) 

true_V_value_record

Generating 5000 input measure samples: 100%|██████████| 5000/5000 [00:26<00:00, 192.04it/s]
Generate source samples for statistical inferences: 100%|██████████| 200/200 [1:51:26<00:00, 33.43s/it]


[5397.046834854326,
 5547.691248695166,
 5405.427283229104,
 5406.308614819565,
 5297.37647299583,
 5402.627246923042,
 5409.724495046458,
 5470.236685714156,
 5289.290019389455,
 5395.296005728323,
 5430.5921564479795,
 5576.59319040281,
 5301.179177581667,
 5414.5582260733045,
 5513.853809568013,
 5357.091009400492,
 5527.497171674511,
 5402.558432172036,
 5409.413330789272,
 5483.892193544031,
 5404.874183583923,
 5501.968958830162,
 5417.303134946396,
 5423.426710102063,
 5356.222608235903,
 5284.825653774887,
 5343.006988725424,
 5351.664235569966,
 5502.829586522538,
 5446.745021722249,
 5371.456809576078,
 5403.767348413851,
 5452.304629185564,
 5369.85773139796,
 5443.521038082248,
 5417.659178963835,
 5477.63227004117,
 5479.816851429925,
 5428.67064891038,
 5413.679762231173,
 5464.706230001922,
 5285.603458535052,
 5404.488988387082,
 5414.072199990986,
 5387.176531459269,
 5386.375541574873,
 5408.651283614557,
 5332.549960816951,
 5425.760490516177,
 5457.684195058462,
 54

In [21]:
lb, ub = np.percentile(true_V_value_record, [2.5, 97.5])
print(f"95% CI for true V value: [{lb}, {ub}]")

95% CI for true V value: [5285.584013416047, 5527.74167646876]


In [23]:
# save the true V values as a pickle file
save_pathname = f"results/entropic_measures_5_seed_1009_samples_5000_dim_2_epsilon_10"
with open(f"{save_pathname}/true_V_values_dim2.pkl", "wb") as f:
    pickle.dump(true_V_value_record, f)

# # Load the true V values
# with open(f"{save_pathname}/true_V_values.pkl", "rb") as f:
#     loaded_true_V_values = pickle.load(f)
